In [63]:
import pandas as pd
import numpy as np

In [64]:
# Establecer semilla para reproducir el experimento
np.random.seed(100)

# Definir la ruta de la base de datos
base_path = "../../BaseLD.xlsx"

# Leer el archivo
data = pd.read_excel(base_path, sheet_name=0).to_numpy().astype(float)
print(data[0])

[ 80. 100.  96. 100. 100.  97.  96.  72.  98.  77.  98.  96.  80.  71.
  98.  69.  77.  81.  98.  83.  64.  66.  72.  80.  78.  91.  79.  97.
  89.  99.]


In [65]:
# Obtener el número de filas y columnas
nrows, ncols = data.shape
print(nrows, ncols)

27751 30


In [66]:
# Definir las neuronas
neuronas = np.array([
    np.min(data, axis=0),
    np.mean(data, axis=0),
    np.max(data, axis=0)])

print(neuronas[0])

[18.  8. 29. 18. 23. 18.  8.  8. 23.  3.  2. 18. 18. 18.  8.  8. -2. 18.
 18.  3.  3.  3.  8.  3.  3. 18.  8. 13.  3.  8.]


In [67]:
# Definir la matriz de pertenencia y el conteo de núcleos
pertenencia = np.zeros(nrows, dtype=int)

for k in range(3):
    cluster_count = np.zeros(3, dtype=int)
    for i in range(nrows):
        fila_actual = data[i]

        # Calcular la distancia a cada neurona
        distancia = np.linalg.norm(neuronas - fila_actual, axis=1)

        # Obtener el índice de la neurona más cercana
        min_index = np.argmin(distancia)

        # Actualizar la neurona y el núcleo correspondientes
        eta = 0.5 * (1 - i / nrows)
        neuronas[min_index] += eta * (fila_actual - neuronas[min_index])

        # Actualizar el conteo de núcleos y la matriz de pertenencia
        cluster_count[min_index] += 1
        pertenencia[i] = min_index

In [68]:
# Exportar neuronas
nombre_neurona = ["Neurona1", "Neurona2", "Neurona3"]
neuronas_df = pd.DataFrame(neuronas, index=nombre_neurona, columns=[f'Variable_{j+1}' for j in range(ncols)])
neuronas_df.reset_index(inplace=True)
neuronas_df.to_csv("./NEURONAS.csv", index=False)

In [69]:
# Exportar suma promedio
suma_promedio = np.mean(neuronas, axis=1)
suma_promedio_df = pd.DataFrame({'Nombre_Neurona': nombre_neurona,
    'Suma_Promedio': suma_promedio})
suma_promedio_df.to_csv("./SUMA_PROMEDIO.csv", index=False)

In [70]:
# Exportar conteo de elementos en cada núcleo
cluster_df = pd.DataFrame({'Nombre_Neurona': nombre_neurona,
    'Elementos_Cluster': cluster_count})
cluster_df.to_csv(f"./TOTAL_ELEMENTOS_CLUSTER.csv", index=False)

In [71]:
# Exportar matriz de pertenencia
base_actualizada = np.column_stack((pertenencia, data))
base_actualizada_df = pd.DataFrame(base_actualizada, columns=['Pertenencia'] \
    + [f'Variable_{i+1}' for i in range(ncols)])
base_actualizada_df.to_csv(f"./PERTENENCIA.csv", index=False)